https://github.com/facebookresearch/faiss/blob/main/tutorial/python/2-IVFFlat.py

In [1]:
import numpy as np
import metrics_utils


In [2]:
import faiss
import pandas as pd

In [3]:
df = pd.read_parquet("gs://np-public-training-temp/stackoverflow/final_subset/posts_with_embedding.parquet")

In [4]:
len(df)

219841

In [5]:
df.head()

,Id,AcceptedAnswerId,Title,QuestionBody,Tags,ViewCount,AnswerCount,CommentCount,Score,CreationDate,AnswerId,AcceptedAnswerBody,embedding
0,15020895,-1,Python int-byte efficient data structure,i am currently storing key-values of type int-...,python data-structures,155,0,3,1,2013-02-22 09:33:26.360,-1,,"[-0.012659057043492794, -0.00613340875133872, ..."
1,68487902,-1,Why does the Variance of Laplace very differen...,TL;DR: How can I use skimage.filters.laplace(i...,python opencv image-processing computer-vision...,391,0,5,1,2021-07-22 15:50:34.220,-1,,"[0.06565584242343903, -0.031634315848350525, 0..."
2,61391327,-1,Why input never ends,I have python 3.7 installed and I have this co...,python python-3.x input,104,1,6,3,2020-04-23 15:43:03.497,-1,,"[0.01758185401558876, -0.07531370967626572, 0...."
3,28852710,-1,Crashes with piecewise linear objective for gu...,We have a complex optimization problem which i...,python crash gurobi piecewise,403,1,1,3,2015-03-04 10:58:16.370,-1,,"[0.010070848278701305, 0.06428138166666031, -0..."
4,24043029,-1,Python TypeError: plotdatehist() got an unexpe...,apologies beforehand if this is a stupid quest...,python typeerror,419,0,7,0,2014-06-04 16:42:32.257,-1,,"[0.034622225910425186, 0.0662810429930687, 0.0..."


In [6]:
df_subset = df.sample(frac=1).head(500_000).reset_index(drop=True) 
df_subset['id_faiss'] = range(0, 0+len(df_subset))

In [7]:
df_subset

,Id,AcceptedAnswerId,Title,QuestionBody,Tags,ViewCount,AnswerCount,CommentCount,Score,CreationDate,AnswerId,AcceptedAnswerBody,embedding,id_faiss
0,71957513,-1,unit tests for a class containing methods doin...,I have a python class where I have a quicksigh...,python unit-testing mocking python-unittest py...,19,0,0,0,2022-04-21 16:01:05.843,-1,,"[0.01994287222623825, 0.13373127579689026, 0.0...",0
1,53981547,53981930,Calculate local clustering coefficient of a ve...,\n\nI found an example showing how to calculat...,r cluster-analysis igraph graph-theory,2255,1,0,3,2018-12-30 21:23:54.897,53981930,All of this can be done in igraph. It is nice ...,"[-0.017001142725348473, 0.03255533054471016, -...",1
2,60900596,-1,ModuleNotFoundError: No module named 'todo_lis...,I am following a tutorial online for a to-do a...,python django,777,1,6,0,2020-03-28 11:35:05.110,-1,,"[0.007443961221724749, -0.0237520020455122, 0....",2
3,34090439,34093079,index.html served by django cant locate socket...,I'm trying to add realtime feature to a page s...,javascript python django node.js sockets,207,1,4,1,2015-12-04 14:20:57.697,34093079,Since you're running Node.js on a separate URL...,"[0.028874576091766357, -0.06416145712137222, 0...",3
4,70950415,-1,How to calculate gaussian bins manually?,A bin—sometimes called a class interval. I hav...,python gaussian,17,0,1,0,2022-02-02 04:11:16.967,-1,,"[-0.030148359015583992, 0.039703480899333954, ...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219836,16543573,16553503,How to reuse / clone an sqlalchemy query,It seems to me like going through the whole pr...,python caching sqlalchemy,4538,2,0,4,2013-05-14 12:46:41.287,16553503,\n It seems to me like going through the whol...,"[0.042191971093416214, 0.08244752138853073, -0...",219836
219837,52081561,52082311,How Pandas read_html can get only selected col...,I'm trying to extract the particular columns f...,python pandas,2640,2,2,1,2018-08-29 15:52:14.677,52082311,to select a subset of columns you could use\n\...,"[0.03314579650759697, 0.12591011822223663, -0....",219837
219838,60624604,-1,Python Logic to find the sum of a sub list sat...,I need to find the sum of the list elements be...,python list,144,6,11,1,2020-03-10 19:25:40.903,-1,,"[-0.0366428941488266, 0.06802389025688171, -0....",219838
219839,29354283,-1,Python/Pygame sprites using same class not upd...,"[CODE]\n\nQuick rundown, shipX should move whe...",python class pygame sprite,240,1,0,1,2015-03-30 19:09:49.567,-1,,"[0.04933905228972435, -0.00944867730140686, 0....",219839


In [8]:
k = 5

emmbedding corpus

In [9]:
xb = df_subset['embedding'].tolist()
xb = np.array(xb).astype('float32')

In [10]:
xb.shape

(219841, 768)

dimension

In [11]:
d = xb.shape[-1]                       
d

768

## Naive Index : flat

In [12]:
index = faiss.IndexFlatL2(d)


In [13]:

index.is_trained


True

In [14]:
index.add(xb)                

vectors to search for

In [15]:
xq = xb

In [16]:
len(xq)

219841

In [17]:
%%time
D, I = index.search(xq, k)     


CPU times: user 1h 3min, sys: 3min 28s, total: 1h 6min 28s
Wall time: 4min 12s


In [18]:
df_res = df_subset[['Title','id_faiss']].copy()
df_res

,Title,id_faiss
0,unit tests for a class containing methods doin...,0
1,Calculate local clustering coefficient of a ve...,1
2,ModuleNotFoundError: No module named 'todo_lis...,2
3,index.html served by django cant locate socket...,3
4,How to calculate gaussian bins manually?,4
...,...,...
219836,How to reuse / clone an sqlalchemy query,219836
219837,How Pandas read_html can get only selected col...,219837
219838,Python Logic to find the sum of a sub list sat...,219838
219839,Python/Pygame sprites using same class not upd...,219839


In [19]:
def calculate_recall( index, target, k):
    
    res = index == target
    
    res = res[:k]
    
    return True in res


def compute_recall( I, df, k):
    
    all_res = []
    for idx, row in enumerate(I):
        res = calculate_recall(row, idx, k)
        all_res.append(res)
        
        
    
    
    return all_res

In [20]:
#df_res['r@1'] = compute_recall(I,df_subset,k=1)

res =   compute_recall(I,df_subset,k=1)

len(res), sum(res)


(219841, 219817)

In [21]:
#calculate_recall (I[1] , 1 , k =1)

In [22]:
#df_res['r@1'].value_counts()

## IVF Index

In [23]:
nlist = 100 # nmber of clusters

quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)


In [24]:
assert not index.is_trained
index.train(xb)
assert index.is_trained

In [25]:
xq = xb

In [26]:
index.add(xb)         


In [27]:
%%time


index.nprobe = 1              # default nprobe is 1

D, I = index.search(xq, k)     # actual search


CPU times: user 10min 6s, sys: 290 ms, total: 10min 6s
Wall time: 38.8 s


In [28]:
df_res['r@1'] = compute_recall(I,df_subset,k=1)
df_res['r@1'].value_counts()

True     219835
False         6
Name: r@1, dtype: int64

In [29]:
%%time

index.nprobe = 5              # default nprobe is 1, max is nlist
D, I = index.search(xq, k)


CPU times: user 47min 58s, sys: 1.01 s, total: 47min 59s
Wall time: 3min 3s


In [30]:
df_res['r@1'] = compute_recall(I,df_subset,k=1)
df_res['r@1'].value_counts()

True     219835
False         6
Name: r@1, dtype: int64

In [31]:
faiss.write_index(index, "/tmp/index_ivf.bin")


### PQ Index

In [32]:
m = 8 # how many subchunks for the vector
nbits = 8  # number of bits per subquantizer, k* = 2**nbits ; number of cluster_ids 
index = faiss.IndexPQ(d, m, nbits)

In [33]:
assert not index.is_trained
index.train(xb)
assert index.is_trained

In [34]:
D, I = index.search(xq, k)


In [35]:
res =   compute_recall(I,df_subset,k=1)

len(res), sum(res)

(219841, 0)

In [36]:
res

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [37]:
faiss.write_index(index, "/tmp/index_pq.bin")

## Space consumed

In [38]:
!ls -lah /tmp/index_*

-rw-r--r-- 1 jupyter jupyter 647M Nov  2 14:18 /tmp/index_ivf.bin
-rw-r--r-- 1 jupyter jupyter 769K Nov  2 14:18 /tmp/index_pq.bin
